In [4]:
import torch
import torchvision.transforms as transforms
import torchvision.models as models
from PIL import Image
import numpy as np
import joblib
import pandas as pd
import os

# 📌 Cargar el dataset
metadata_path = "metadata.xlsx"

if not os.path.exists(metadata_path):
    print(f"⚠️ No se encontró el archivo '{metadata_path}'. Verifica su ubicación.")
    exit()

df = pd.read_excel(metadata_path)

# 📌 Cargar modelo de extracción de features
model = models.resnet18(pretrained=True)
model = torch.nn.Sequential(*list(model.children())[:-1])  # Quitar la última capa
model.eval()

# 📌 Cargar modelo KNN
knn_path = "models/knn_model.pkl"

if not os.path.exists(knn_path):
    print(f"⚠️ No se encontró el modelo '{knn_path}'. Asegúrate de entrenarlo primero.")
    exit()

knn = joblib.load(knn_path)

# 📌 Transformaciones para la imagen
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 📌 Clasificar una nueva imagen
img_path = "imagen_prueba.jpg"

if not os.path.exists(img_path):
    print(f"⚠️ No se encontró la imagen '{img_path}'. Verifica la ruta.")
    exit()

image = Image.open(img_path).convert("RGB")
image = transform(image).unsqueeze(0)

# 📌 Extraer características con ResNet18
with torch.no_grad():
    embedding = model(image).flatten().numpy()

# 🔍 Predecir con KNN
predicted_label = knn.predict([embedding])[0]

# 📌 Buscar información en el dataset
resultado = df[df["imagen"] == predicted_label]

if not resultado.empty:
    señal = resultado.iloc[0]["señal"]
    tipo = resultado.iloc[0]["tipo"]
    descripcion = resultado.iloc[0]["descripcion"]
    
    print(f"🚦 La señal detectada es: {señal}")
    print(f"📌 Tipo: {tipo}")
    print(f"ℹ️ Descripción: {descripcion}")
else:
    print("⚠️ No se encontró información para la señal detectada.")


🚦 La señal detectada es: SR-01
📌 Tipo: Reglamentaria
ℹ️ Descripción: Pare


c:\Users\wotan\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\wotan\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
